#### <font color='crimson'> Mount to Google Drive </font>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### <font color='crimson'> Import Python Libraries </font>

In [ ]:
import numpy as np
import pandas as pd
import pandas as pd
import regex as re
import spacy
from sklearn.feature_extraction.text import CountVectorizer

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

import warnings
warnings.simplefilter('ignore')

from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, RepeatVector
from keras.preprocessing.text import Tokenizer
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from keras import optimizers

from tqdm.notebook import tqdm
tqdm.pandas()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### <font color='crimson'> Step 1: Import and merge all the three datasets </font>

In [ ]:

def read_file(filepath):
    with open(filepath,"r",encoding='utf-8') as f:
        file = f.readlines()
    return file


def read_all_files(filepaths):
    complete_df = {}
    for f in filepaths:
        de = read_file('drive/MyDrive/colab_notebook/great-learning-projects/nlp-language-translation/data/'+f[0])
        en = read_file('drive/MyDrive/colab_notebook/great-learning-projects/nlp-language-translation/data/'+f[1])

        df = pd.DataFrame([de, en]).T
        df.columns = ['german', 'english']

        if len(complete_df)==0:
            complete_df = df.copy()
        else:
            complete_df = pd.concat([complete_df, df])
    return complete_df


filepaths = [
                ['news-commentary-v9.de-en.de', 'news-commentary-v9.de-en.en'],
                #['europarl-v7.de-en.de','europarl-v7.de-en.en'],
                #['commoncrawl.de-en.de','commoncrawl.de-en.en']
            ]

df = read_all_files(filepaths)
df = df.head(10000) # For faster model training, Use complete for final report
print(df.shape)
df.head()




(10000, 2)


,german,english
0,Steigt Gold auf 10.000 Dollar?\n,"$10,000 Gold?\n"
1,"SAN FRANCISCO – Es war noch nie leicht, ein ra...",SAN FRANCISCO – It has never been easy to have...
2,In letzter Zeit allerdings ist dies schwierige...,"Lately, with gold prices up more than 300% ove..."
3,Erst letzten Dezember verfassten meine Kollege...,"Just last December, fellow economists Martin F..."
4,"Und es kam, wie es kommen musste.\n",Wouldn’t you know it?\n


#### <font color='crimson'> Step 2: Data cleansing </font>
- Drop Null Values
- Remove Leading and Trailing spaces
- Convert to lower case
- Keep only alphabets by removing special characters and numbers
- Remove English and German stopwords
- Drop Duplicate Records

In [ ]:
df.dropna(inplace=True)
print(df.shape)

(10000, 2)


In [ ]:
df['german']  = df['german'].progress_apply(lambda x: str(x).strip())
df['english'] = df['english'].progress_apply(lambda x: str(x).strip())

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

In [ ]:
for c in tqdm(df.columns):
    df[c] = df[c].progress_apply(lambda x: str(x).lower())

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

In [ ]:
for c in tqdm(df.columns):
    df[c] = df[c].progress_apply(lambda x: re.sub(r'[^a-zA-Z ]+', '', str(x)))

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

In [ ]:
df.head()

,german,english
0,steigt gold auf dollar,gold
1,san francisco es war noch nie leicht ein rati...,san francisco it has never been easy to have ...
2,in letzter zeit allerdings ist dies schwierige...,lately with gold prices up more than over the...
3,erst letzten dezember verfassten meine kollege...,just last december fellow economists martin fe...
4,und es kam wie es kommen musste,wouldnt you know it


In [ ]:
#german_stop_words = stopwords.words('german')
#english_stop_words = stopwords.words('english')

#df['german'] = df['german'].progress_apply(lambda x: ' '.join([i for i in str(x).split(' ') if i not in german_stop_words]))
#df['english'] = df['english'].progress_apply(lambda x: ' '.join([i for i in str(x).split(' ') if i not in english_stop_words]))

In [ ]:
df.drop_duplicates(inplace=True)
print(df.shape)
display(df.head())

(9926, 2)


,german,english
0,steigt gold auf dollar,gold
1,san francisco es war noch nie leicht ein rati...,san francisco it has never been easy to have ...
2,in letzter zeit allerdings ist dies schwierige...,lately with gold prices up more than over the...
3,erst letzten dezember verfassten meine kollege...,just last december fellow economists martin fe...
4,und es kam wie es kommen musste,wouldnt you know it


#### <font color='crimson'> Step 3: NLP pre processing - Dataset suitable to be used for AIML model learning </font>

- Tokenize English &  German

In [ ]:
def get_tokenization(data):

  def tokenization(data):
    t = Tokenizer()
    t.fit_on_texts(data)
    return t

  t = tokenization(data)
  vocab_size = len(t.word_index) + 1
  length = 8
  return t, vocab_size, length

english_tokenizer, english_vocab_size, english_length = get_tokenization(df['english'])
german_tokenizer, german_vocab_size, german_length    = get_tokenization(df['german'])


In [ ]:
print('English vocab size and length:', english_vocab_size, english_length)
print('German  vocab size and length:', german_vocab_size, german_length)

English vocab size and length: 16335 8
German  vocab size and length: 26568 8


- Sequence and padding - for equal length

In [ ]:
def encode_to_sequence(tokenizer, data, length):
  seq = tokenizer.texts_to_sequences(data)
  seq = pad_sequences(seq, maxlen=length, padding='post')
  return seq


In [ ]:
english_sequence = encode_to_sequence(english_tokenizer, df['english'], 8)
german_sequence  = encode_to_sequence(german_tokenizer, df['german'], 8)

english_sequence.shape, german_sequence.shape

((9926, 8), (9926, 8))

#### <font color='crimson'> Step 4: Design, train and test simple RNN & LSTM model </font>

- Train Test Split - for model evaluation

In [ ]:
train_data_size = int(0.8*len(df))

train_english_data_as_x, train_german_data_as_y = english_sequence[:train_data_size], german_sequence[:train_data_size]
test_english_data_as_x, test_german_data_as_y = english_sequence[train_data_size:], german_sequence[train_data_size:]

print('Train Data =', len(train_english_data_as_x), len(train_german_data_as_y))
print('Test  Data =', len(test_english_data_as_x), len(test_german_data_as_y))


Train Data = 7940 7940
Test  Data = 1986 1986


- LSTM Model Definition

In [ ]:
def build_model(input_vocab_size, output_vocab_size, input_length, output_length, units):

  model = Sequential()
  model.add(Embedding(input_vocab_size, units, input_length=input_length, mask_zero=True))
  model.add(LSTM(units))
  model.add(RepeatVector(output_length))
  model.add(LSTM(units, return_sequences=True))
  model.add(Dense(output_vocab_size, activation='softmax'))
  return model

model = build_model(english_vocab_size, german_vocab_size, german_length, english_length, 512)


- Setting up Optimizer

In [ ]:
optimizer_setting = optimizers.RMSprop(learning_rate=0.001)

- Model Compilation

In [ ]:
model.compile(optimizer=optimizer_setting, loss='sparse_categorical_crossentropy')

- Model Training

In [ ]:
g_shape = train_german_data_as_y.shape
reshaped_train_german_data_as_y = train_german_data_as_y.reshape(g_shape[0], g_shape[1],1)

# 2 epochs are used to get results faster, epoch = 10 will be used in final for better results
actual_fits = model.fit(train_english_data_as_x, reshaped_train_german_data_as_y, epochs=2, validation_split=0.2)


Epoch 1/2
199/199 [==============================] - 270s 1s/step - loss: 8.1494 - val_loss: 7.7476
Epoch 2/2
185/199 [==========================>...] - ETA: 16s - loss: 7.6448

**Please note**:

Loss is higher as data volume used for model training is significantly low i.e. 10K rows from 1 dataset only with number of epochs reduced to 2 to obtain results faster.

For actual and final execution, all datasets and rows will be considered with epochs=10 and using batch size. That should reduce the loss from 7.6 significantly lower.

- Model Prediction

In [ ]:
test_english_data_as_x[:2]

array([[ 2047,   416,    71,   700,     5,     1, 14933,   146],
       [    2,  1090,   142,     6,  2047,   416,    22,  1201]],
      dtype=int32)

In [ ]:
e_shape = test_english_data_as_x.shape
reshaped_test_english_data_as_x = test_english_data_as_x.reshape(e_shape[0], e_shape[1],1)

prediction = model.predict(reshaped_test_english_data_as_x)
classes_x  = np.argmax(prediction,axis=1)


63/63 [==============================] - 20s 321ms/step


- Get Model Prediction (Numbers to Words) for comparison

**Please note:**

Since dataset volumn is significantly low and epochs=2, the language translation model will be inferior. Thus, actual vs prediction is not shown below. Reason for choosing smaller dataset is longer model training time, 15-20 Hrs and kernal failure on local machine multiple times.

Also, after using complete data volume from all 3 datasets (or atleast 1 dataset) and sufficient epochs, the actual vs prediction comparison will be meaningful.

In [ ]:
def convert_numbers_to_words(tokenizer, i):
  for word, index in tokenizer.word_index.items():
    if index==i:
      return word
  return ''

prediction_df = []
for integers in tqdm(classes_x[:10]): # Testing only for first 10 entries
  german_prediction = [convert_numbers_to_words(german_tokenizer, j) for j in integers]
  prediction_df.append(' '.join(german_prediction))


  0%|          | 0/10 [00:00<?, ?it/s]

#### <font color='crisom'> Step 5: Interim report </font>

In [ ]:
# Provided as PPT